In [1]:
import numpy as np
import cv2
import PIL
import glob
import csv
from keras.models import load_model
from PIL import Image, ImageOps, ImageChops

Using TensorFlow backend.
//anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
//anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
//anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
//anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: Future

In [2]:
#Import model
model_path = 'foldmodel2.h5' 
model_weights_path = 'foldmodelweights2.h5'
#Load model
model = load_model(model_path) 
model.load_weights(model_weights_path)






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [3]:
#Membuat Kamus
f = open('/Users/server/Desktop/kamusbatak.csv','r')
reader = csv.reader(f)
kamusbatak = {}
for row in reader:
    kamusbatak[row[0]] = row[1]

f = open('/Users/server/Desktop/kamusminang.csv','r')
reader = csv.reader(f)
kamusminang = {}
for row in reader:
    kamusminang[row[0]] = row[1]
    
f = open('/Users/server/Desktop/kamussunda.csv','r')
reader = csv.reader(f)
kamussunda = {}
for row in reader:
    kamussunda[row[0]] = row[1]

In [4]:
kamusbatak

{'abu': 'Orbuk',
 'air': 'Aek',
 'akar': 'Urat',
 'anak': 'Anak',
 'angin': 'Alogo',
 'anjing': 'Biang',
 'apa': 'Aha',
 'api': 'Api',
 'asap': 'Timus',
 'awan': 'Ombun',
 'ayah': 'Amang/Ama',
 'bagaimana': 'Beha',
 'bagus': 'Bagak',
 'baik': 'Denggan',
 'bakar': 'Tutung',
 'balik': 'Balik/Pudi',
 'banyak': 'Godang',
 'baring': 'Peak',
 'baru': 'Baru/Imbaru',
 'basah': 'Bornok/Tonu/Maraek',
 'batu': 'Batu',
 'beberapa': 'Pigapiga',
 'benar': 'Sintong/Toho',
 'bengkak': 'Bongkak',
 'benih': 'Boni',
 'berat': 'Borat',
 'berburu ': 'Mamuru',
 'berdiri ': 'Jongjong',
 'berenang': 'Marlange',
 'beri': 'Lehon',
 'berjalan': 'Mardalan',
 'berkata ': 'Marhata',
 'berkelahi ': 'Marbada',
 'besar': 'Balga',
 'bilamana': 'Atik Beha',
 'binatang': 'Binatang/Dorbia',
 'bintang': 'Bintang',
 'buah': 'Parbue/Boras',
 'bulan': 'Bulan',
 'bulu': 'Bulu',
 'bunga': 'Bunga',
 'bunuh': 'Pamate/Bunu',
 'buruk': 'Buruk/Roa',
 'burung': 'Pidong',
 'busuk': 'Busuk',
 'cacing': 'Gilok',
 'cium': 'Umma',
 'cuci'

In [17]:
#Fungsi menggabungkan karakter
def concatenate(list):
    result= ''
    for element in list:
        result += str(element)
    return result

In [18]:
hasil = []

for filename in glob.glob("/Users/server/Documents/Data Uji/Device 1/F1_D1/*.jpg"):
    teks = []
    teks.append(filename)
    citra = cv2.imread(filename)
    if citra is None:
        print("Citra Gaada")
        exit()
    #Flip citra untuk menghiondari kesalahan pelabelan CCA
    citra = cv2.transpose(citra)
    citra = cv2.flip(citra, 1)
    #Konversi citra ke grayscale dan biner
    abuAbu = cv2.cvtColor(citra, cv2.COLOR_BGR2GRAY)
    ambang, citraBiner = cv2.threshold(abuAbu, 105, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    #Pelabelan pada objek latarDepan dengan CCA
    jumObjek, label = cv2.connectedComponents(citraBiner)
    #latarBelakang dilabeli 0, dan tiap objek dengan 1,2,dst
    label = label + 1
    #Penerapan watershed pada tiap label(objek) hasil CCA
    label = cv2.watershed(citra, label)
    citraBiner = cv2.cvtColor(citraBiner,cv2.COLOR_GRAY2RGB)
    tinggi, lebar = citraBiner.shape[:2]
    #Segmentasi karakter
    for indeks in range(1, jumObjek):
    #         d = 0
        mask = np.zeros((tinggi, lebar), np.uint8)
        mask[label == indeks + 1] = 1
        objek = citraBiner * mask[:, :, np.newaxis]
        objek = objek[..., : : -1]
        objek = cv2.transpose(objek)
        objek = cv2.flip(objek, 0)
        objek = Image.fromarray(objek)#convert cv2 to PIL image
        thresh = objek.point(lambda p: p < 25 and 255)
        # Get bounding box of thresholded image
        bbox1 = thresh.getbbox()
        crop1 = thresh.crop(bbox1)
        # Invert and crop again
        crop1n = ImageOps.invert(crop1)
        bbox2  = crop1n.getbbox()
        crop2  = crop1.crop(bbox2)  # this just for debug
        # Trim original, unthresholded, uninverted image to the two bounding boxes
        result = objek.crop(bbox1).crop(bbox2)
        img_w, img_h = result.size
        background = Image.new('RGB', (32, 32), (0, 0, 0, 0))
        bg_w, bg_h = background.size
        offset = ((bg_w - img_w) // 2, (bg_h - img_h) // 2)
        background.paste(result, offset)
        new = background

        from numpy import array
    #     new = Image.open("new.jpg")
        thresh = new.point(lambda p: p < 25 and 255) # invert citra menjadi berlatarbelakang putih dan objek hitam
        thresh = thresh.convert('L') #convert citra menjadi grayscale
        binaryArray = array(thresh.getdata()).reshape(-1,1).T
        x = binaryArray.reshape(1,32,32,1)
        x = x.astype('float32')
        x /= 255
        array = model.predict(x)
        result = array[0]
        #print(result)
        answer = np.argmax(result)
        if answer == 0:
            answer = "a"
        elif answer == 1:
            answer = "b"
        elif answer == 2:
            answer = "c"
        elif answer == 3:
            answer = "d"
        elif answer == 4:
            answer = "e"
        elif answer == 5:
            answer = "f"
        elif answer == 6:
            answer = "g"
        elif answer == 7:
            answer = "h"
        elif answer == 8:
            answer = "i"
        elif answer == 9:
            answer = "j"
        elif answer == 10:
            answer = "k"
        elif answer == 11:
            answer = "l"
        elif answer == 12:
            answer = "m"
        elif answer == 13:
            answer = "n"
        elif answer == 14:
            answer = "o"
        elif answer == 15:
            answer = "p"
        elif answer == 16:
            answer = "q"
        elif answer == 17:
            answer = "r"
        elif answer == 18:
            answer = "s"
        elif answer == 19:
            answer = "t"
        elif answer == 20:
            answer = "u"
        elif answer == 21:
            answer = "v"
        elif answer == 22:
            answer = "w"
        elif answer == 23:
            answer = "x"
        elif answer == 24:
            answer = "y"
        elif answer == 25:
            answer = "z"
        teks.append(answer)

    teks = concatenate(teks)
    hasil.append(teks)
    
hasil

['/Users/server/Documents/Data Uji/Device 1/F1_D1/T72_F1_D1.jpggosok',
 '/Users/server/Documents/Data Uji/Device 1/F1_D1/T104_F1_D1.jpgkepala',
 '/Users/server/Documents/Data Uji/Device 1/F1_D1/T110_F1_D1.jpgkuniing',
 '/Users/server/Documents/Data Uji/Device 1/F1_D1/T66_F1_D1.jpggalii',
 '/Users/server/Documents/Data Uji/Device 1/F1_D1/T131_F1_D1.jpgmatii',
 '/Users/server/Documents/Data Uji/Device 1/F1_D1/T47_F1_D1.jpgciium',
 '/Users/server/Documents/Data Uji/Device 1/F1_D1/T182_F1_D1.jpgtariik',
 '/Users/server/Documents/Data Uji/Device 1/F1_D1/T29_F1_D1.jpgberenang',
 '/Users/server/Documents/Data Uji/Device 1/F1_D1/T196_F1_D1.jpgtulang',
 '/Users/server/Documents/Data Uji/Device 1/F1_D1/T1_F1_D1.jpgabu',
 '/Users/server/Documents/Data Uji/Device 1/F1_D1/T53_F1_D1.jpgdatang',
 '/Users/server/Documents/Data Uji/Device 1/F1_D1/T125_F1_D1.jpglutut',
 '/Users/server/Documents/Data Uji/Device 1/F1_D1/T173_F1_D1.jpgsuamii',
 '/Users/server/Documents/Data Uji/Device 1/F1_D1/T109_F1_D1.jp

### PENERJEMAHAN

In [8]:
#Algoritma Levensthein Distance
def levdist(s1, s2):
    if len(s1) > len(s2):
        s1, s2 = s2, s1

    jarak = range(len(s1) + 1)
    for i2, c2 in enumerate(s2):
        jarak1 = [i2+1]
        for i1, c1 in enumerate(s1):
            if c1 == c2:
                jarak1.append(jarak[i1])
            else:
                jarak1.append(1 + min((jarak[i1], jarak[i1 + 1], jarak1[-1])))
        jarak = jarak1
    return jarak[-1]

In [13]:
#Fungsi untuk menerjemahkan kata hasil klasifikasi
def dist1(teks):
    nol = ['Nol']
    satu = ['Satu']
    dua = ['Dua']
    tiga = ['Tiga']
    empat = ['Empat']
    lima = ['Lima']
    enam = ['Enam']
    tujuh = ['Tujuh']
    delapan = ['Delapan']
    sembilan = ['Sembilan']
    sepuluh = ['Sepuluh']
    none = ['None']
    for x, y in kamusbatak.items():
        a = levdist(teks,x)
        if a == 1:
            satu.append(x)
            satu.append(y)
        elif a == 2:
            dua.append(x)
            dua.append(y)
        elif a == 3:
            tiga.append(x)
            tiga.append(y)
        elif a == 4:
            empat.append(x)
            empat.append(y)
        elif a == 5:
            lima.append(x)
            lima.append(y)
        elif a == 6:
            enam.append(x)
            enam.append(y)
        elif a == 7:
            tujuh.append(x)
            tujuh.append(y)
        elif a == 8:
            delapan.append(x)
            delapan.append(y)
        elif a == 9:
            sembilan.append(x)
            sembilan.append(y)
        elif a == 10:
            sepuluh.append(x)
            sepuluh.append(y)
        elif a == 0:
            nol.append(x)
            nol.append(y)
        else:
            none.append(teks)
    if len(nol)>=3:
        print(teks,',',nol[1],',','0',',',nol[2])
    elif len(satu)>=3:
        print(teks,',',satu[1],',','1',',',satu[2])
    elif len(dua)>=3:
        print(teks,',',dua[1],',','2',',',dua[2])
    elif len(tiga)>=3:
        print(teks,',',tiga[1],',','3',',',tiga[2])
    elif len(empat)>=3:
        print(teks,',',empat[1],',','4',',',empat[2])
    elif len(lima)>=3:
        print(teks,',',lima[1],',','5',',',lima[2])
    elif len(enam)>=3:
        print(teks,',',enam[1],',','6',',',enam[2])
    elif len(tujuh)>=3:
        print(teks,',',tujuh[1],',','7',',',tujuh[2])
    elif len(delapan)>=3:
        print(teks,',',delapan[1],',','8',',',delapan[2])
    elif len(sembilan)>=3:
        print(teks,',',sembilan[1],',','9',',',sembilan[2])
    elif len(sepuluh)>=3:
        print(teks,',',sepuluh[1],',','10',',',sepuluh[2])
    else:
        print(teks,',',none[0],',','None',',','None')

In [15]:
F1_D1 = ['abu','aiir','akar','anak','angiin','anjiiing','apa','apii','asap','awan','ayah','bagaiimana','bagus','baiik','bakar','baliik','banyak','bariing','baru','basah','batu','beberapa','benar','bengkak','beniih','berat','berburu','berdiirii','berenang','berii','berjialan','berkata','berkelahii','besar','biilamana','biinatang','biintang','buah','builan','bulu','bunga','bunuh','buruk','burung','busuk','caciing','ciium','cucii','dagiing','dan','danau','darah','datang','daun','debu','dekat','dengan','dengar','diingiin','dorong','dua','duduk','ekor','empat','engkau','galii','garam','garuk','gemuk','giigii','giigiit','gosok','gunung','hantam','hapus','hatii','hiidung','hiidup','hiijiau','hiisap','hiitam','hiitung','hujian','hutan','la','lbu','lkan','lkat','lnii','lsterii','ltu','jahiit','jantung','jatuh','jauh','kabut','kakii','kalau','kamli','kamu','kanan','orena','keciil','kepala','keriing','kiirii','kotor','kuku','kuliit','kuniing','kutu','laiin','langiit','laut','lebar','leher','lelakii','lempar','liiciin','liidah','liihat','liima','ludah','lurus','lutut','maiin','makan','malam','mata','mataharii','matii','membelah','mengaliir','mengapung','meniikam','merah','mereka','miinum','mulut','muntah','nama','nantii','napas','nyanyii','orang','pada','panas','panjiang','pasiir','pegang','pendek','peras','perempuan','pergii','perut','piikiir','pohon','potong','punggung','pusar','puliih','rambut','rumput','sakiit','satu','saya','sayap','sediikiit','sempiit','semua','siiang','siiapa','suamii','sungaii','tahu','tahun','tajiam','takut','talii','tanah','tangan','tariik','tebal','teliinga','telur','terbang','tedawa','tetek','tlidak','tiidur','tiiga','tiipiis','tiiup','tongkat','tua','tulang','tumpul','uang','ular','usus']
F2_D1 = ['abu','mir','li','ains','agin','mjiiiiig','apa','apli','mizip','avmi','ayidi','ibgaiinaitlai','baigiiisi','ibaiiik','bau','bqilri','boiyaiki','biatimri','ibaru','basiah','ibatu','bcbcirqa','bqinia','bciinas','bcinhi','berat','rrburu','bqirkliirli','berenmsr','miiiti','bqirjialain','beirkaitzi','berikeldii','beisai','ibiilaiiinaiinai','bmiaiuinsr','bmitoipi','bmii','ibulan','buii','blinpi','bunuh','biirla','burilw','blsiik','ciaiqitingi','tiiiiuni','czucii','ljagiiw','ljoi','dainaiii','dadi','ijamsr','daiun','ijcbii','ijemti','denm','ljqivira','ljiinpin','dorowi','duaii','ijuqiil','lixior','cimpait','fitigkau','cridti','craiaitl','giaw','tiieiinuk','ciiigiii','gisiriit','cqisok','crilinumr','hotain','haipuisi','haitii','hiirlungi','hiirllip','hiijiaiii','hiiisizipi','hiitaiin','hiititinsr','hlijim','hunn','ilzi','iljii','lmi','luit','inii','isitqvrii','ltiu','jmiiiti','jantung','lamh','jailili','kabuti','kmii','kalmi','oitnti','qinlli','mioi','kairqiinai','kicciil','kema','kciriiiilg','oiii','kotior','ktiku','kuliit','kuniiitgi','kiitiu','limi','misiriiti','lvaiitt','mbc','leher','lellii','mmpoi','liiciiin','liiildi','lhiati','lliiina','luqld','lurlis','lutltt','nami','maikati','ndatit','matai','nzimiairii','natii','meinbelmi','menmtir','meiippungi','nicniikatn','nqirdi','mercikaii','niinltm','miilut','munm','inatiai','nantti','napaisi','nyainyii','oraivr','lzaiida','poiziisi','izoijioisir','paisci','pegmig','pendqvk','perais','perqinpuain','ivqirsirii','ivqvriiii','lvmiir','lvohon','potiom','punww','piiisiiair','piiiiiih','raitlbiit','rmnplit','isiaiiti','isatu','isaja','vsiaiyap','sidmiit','vqtnljiiti','iqismiiaii','qsiiiavtg','vsilialai','isumiii','islinsifai','tialiii','qidun','taijiiaijtt','niaikuiii','tjiii','tmiaih','toigoi','liariik','riielid','tcilmiai','tqilqir','liqirbaiigi','iiertiaiwai','qierk','tiidsi','qiiiqllc','tiigiai','tiiijiiisii','liiiup','tomkait','riiuai','iiulaitifi','rliutnljul','uang','ulair','uisiusi']
F3_D1 = ['abu','aiir','akar','anak','angiin','anjiiing','apa','apii','asap','awan','ayah','aagaiimana','bagus','saiik','sakar','aaliik','banyak','bariing','baru','basah','batu','seberapa','aenar','bengkak','beniih','berat','berburu','berdiirii','berenang','berii','berjialan','serkata','berkelahii','besar','biilanana','biinatang','aiintang','buah','bulan','bulu','bunga','aunuh','buruk','aurune','busuk','caciing','ciiun','cucii','oagiing','oan','oanau','oarah','oatane','oaun','oebu','oekat','oengan','oengar','oiingiin','oorong','uua','ouduk','ekor','empat','engkau','galii','earam','earuk','eenuk','giieii','giigiit','gosok','eunung','hantan','hapus','hatii','hiidung','hiidup','hiiiiau','hiisap','hiitam','hiitung','hujian','hutan','la','lbu','lkan','lkat','lnii','lsterii','ltu','jahiit','jantung','jatuh','jauh','kabut','kakii','kolau','kanii','kanu','kanan','karena','keciil','kepala','keriing','kiirii','kotor','kuku','kuliit','kuniing','kutu','laiin','langiit','laut','lebar','leher','lelakii','lempar','liiciin','liidah','liihat','liina','ludah','lurus','lutut','maiin','makan','malam','mata','mataharii','matii','membelah','menealiir','meneapung','meniikan','merah','mereka','miinum','mulut','muntah','nama','nantii','napas','nyanyii','orang','pada','panas','panjiang','pasiir','fegang','pendek','peras','perempuan','pergii','perut','piikiir','pohon','potong','punggung','pusar','futiih','ranbut','rumput','sakiit','satu','saya','sayap','sediikiit','senpiit','semua','siiang','siiapa','suamii','sungaii','tahu','tahun','tajian','takut','talii','tanah','tangan','tariik','tebal','teliingz','telur','terbang','tenawa','tetek','riidak','tiidur','tiiga','tiipiis','tiiup','tongkat','rua','tulang','tumpul','uang','ular','usus']
F4_D1 = ['abu','aiir','akar','anak','angiin','aniiiing','apa','apli','asap','awan','ayah','bagaiimana','bagus','baiik','bakar','baliik','banyak','bariing','baru','basah','batu','beberapa','benar','bengkak','benlih','berat','berburu','berdiirii','berenang','berii','berjialan','berkata','berkelahli','besar','blilamana','biinatang','biintang','buah','bulan','bulu','bunga','bunuh','buruk','burung','busuk','cacling','ciium','cucli','dagling','dan','danau','darah','datang','dauni','debu','dekat','dengan','dengar','dlinglin','dorong','dua','duduk','ekor','empat','engkau','galii','garam','garuk','gemuk','gligli','gliglit','gosok','gunung','hantam','hapus','hatii','hiidung','hiidup','hliliau','hiisap','hiitam','hlitung','hujian','hutan','la','lbu','lkan','lkat','lnii','lsterii','ltu','jahiit','jantung','jatuh','jauh','kabut','kakii','kalau','kamli','kamu','kanan','karena','keciil','kepala','kerling','klirli','kotor','kukui','kuliit','kunling','kutu','laiin','langiit','laut','lebar','leher','lelakii','lempar','liiclin','llidah','llihat','llima','ludah','lurus','lutut','malin','makan','malam','mata','mataharil','matii','membelah','mengaliir','mengapung','meniikam','merah','mereka','mlinum','mulut','muntah','nama','nantii','napas','nyanyli','orang','pada','panas','paniiang','pasiir','pegang','pendek','peras','perempuan','pergii','perut','pliklir','pohon','potong','punggung','pusar','putiih','rambut','rumput','sakiit','satu','saya','sayap','sedliklit','sempiit','semua','siiang','siiapa','suamli','sungaii','tahu','tahun','tajiam','takut','talii','tanah','tangan','tarlik','tebal','tellinga','telur','terbang','tenawa','tetek','tiidak','tlidur','tiiga','tlipiis','tiiup','tongkat','tua','tulang','tumpul','uang','ular','usus']
F5_D1 = ['abu','atir','ako','anak','angiin','anjiiing','apa','apti','asap','awm','ayd','bagatimana','bagus','baik','bsa','baliik','bmya','bariing','bam','bsah','bam','beberapa','benar','benas','beniih','berat','berbum','berdiirii','berenmg','berti','berjialm','berkata','berkelmiti','bess','biilmma','biinatmg','biinmg','bm','bulan','bulu','bunga','bunuh','buruk','bwung','busuk','caqiing','ciium','cucii','dagiing','do','dmau','darm','datog','dauo','debu','dekat','dengm','denga','dtingiin','dorong','dua','duduk','ekor','empat','engkau','galii','cam','camk','gqmuk','oiigii','gtigtit','gosok','gunung','hmtm','hapus','hatii','hiidung','hiidup','hiijisu','fliisap','hiitm','hiitung','hujim','hum','ia','ibiu','ikm','ikat','inti','isterii','itu','jdiit','jmtung','jamh','jauh','kabut','kakii','kalau','kmii','kamu','kom','orcna','keciil','kepala','kerting','ktirii','kotor','kuku','kuliit','kuniinig','kum','laiin','lmgiit','laut','leba','leher','lelakii','lempai','liiciin','liidm','liihat','liima','ludm','luns','lutut','maiin','mam','malm','mata','matmcii','matti','membeld','menotir','mengapung','meniikm','merd','mereka','miinum','mulut','muntm','nama','nantti','napas','nyoyii','ormg','pada','panas','panjiog','psiir','pegang','pendek','pems','perempuo','pergti','pent','piikiir','pohon','potong','punggung','pusa','putiih','rmbut','rumput','sakiit','satu','saya','sayap','sediiktit','semptit','semua','siiang','siiapa','sumii','sungaii','tmu','tahun','tajiam','takut','talti','tod','tmgm','tciik','tebal','teliinsa','telw','terbang','tedawa','tetek','tiidw','tiidur','tiiga','tiiptis','tiiup','tongkat','tua','tulmg','tumpul','umg','ula','usus']

F1_D2 = ['qu','air','kr','anak','mgiin','anjiiing','apa','qii','map','awan','ayah','bagaiimana','bagus','baiik','bamr','baliik','banyak','bariing','baru','basah','batu','beberapa','benar','bengkak','benlih','berat','berburu','berdlirli','berenang','berli','berjialan','berkata','berkelahii','besar','blilamana','blinatang','blintang','buah','bulan','bulu','bunga','bunuh','buruk','burung','busuk','cacling','ciium','cucii','dagling','dan','danau','darah','damng','daun','debu','demt','xngan','xngar','dlingiin','dorong','dua','duduk','ekor','empat','engkau','galli','garam','garuk','gemuk','gligli','giigiit','gosok','gunung','hantam','hapus','hatli','hiidung','hlidup','hlijiau','hiisap','hlitam','hlitung','hujian','hutan','la','lbu','lkan','lkat','lnli','lsterii','ltu','jahiit','jantung','jatuh','jauh','kabut','kakii','kalau','kamli','kamu','kanan','karena','keclil','kepala','keriing','klirii','kotor','kuku','kullit','kunling','kutu','laiin','langiit','laut','lebar','leher','lelakii','lempar','lliclin','llidah','liihat','llima','ludah','lurus','lutut','malin','makan','malam','mata','mataharli','matli','membelah','mengaliir','mengapung','meniikam','merah','mereka','miinum','mulut','muntah','nama','nantli','napas','nyanyli','orang','pada','panas','panjiang','paslir','pegang','pendek','peras','perempuan','pergli','perut','plikiir','pohon','potong','punggung','pusar','putiih','rambut','rumput','saklit','satu','saya','sayap','sediiklit','sempiit','semua','sliang','sliapa','suamli','sungali','tahu','tahun','tajiam','takut','talii','tanah','tangan','tarlik','tebal','tellinga','telur','terbang','tenawa','tetek','tiidak','tlidur','tiiga','tiipiis','tliup','tongkat','tua','tulang','tumpul','uang','ular','usus']
F2_D2 = ['abti','zilir','ziwiai','iqnsi','iimsnitt','iimljiiiitg','iqiljai','ialjii','iwialj','awm','zqrnid','bgmiana','miemlisi','bmiii','bmi','bdiik','bmrvis','bciting','baiii','baiisiailt','baitu','rriraipa','mnn','bensiqwi','benhi','mrait','bcirljuru','berdriii','bcircinang','bcvrii','rvrjijiain','mrkmii','beirkeimiiii','bieisai','biilamaina','lziiiiaitziilsr','ljiiiiiainsr','biiaili','bztiljiii','biiilll','blinsra','blinlili','buruk','lzurijnsir','lzliisitik','fziiaiciiing','fiviiiuitl','fviuciii','ljoiiriiilgi','ljain','ljaiitaiu','dm','dming','ijaiun','ljcilju','ijcikai','ijcvngm','dengiui','ljiinsiriin','ljijriznsr','ljuai','ljuiliik','liiikkzr','liiiitlijzit','iiiinsrkjiti','fiijiii','fiiairaiiti','iiiziiiiik','tziiiiiiilik','fziiisrii','iiiiisriil','iisroisiiik','fsitliitiitlsr','iiaiititiaiiii','hailvtlisi','haitii','hiiiltittg','htidtilj','hlijiaitl','htiisiaip','hiitaitt','hiitittng','htljialt','huuiin','la','lbu','ljii','lkat','lnli','lisiterti','ltu','jjiliit','iaintiunsri','jatiult','iiaitih','kijilllil','kaikiii','kiailaili','kjiillii','kaiiilll','kjiiiiain','kaircina','kiqisiciiil','kcliidai','kqsiriing','oitili','kotor','ktlkiu','ktiiliiii','ktmting','kliitti','luitl','lmgiit','laiuti','lcsbair','hiltciti','lvcsilwiii','lcsitiljui','liiciiin','liviiqljill','iiiilijit','lviiiiiai','lviitlaill','lviiiiiiisi','ltiiilii','maiiiit','maikiaiti','mailaiiti','maitai','mjitaiiltaiiiii','maitii','niqsittljcilaiiit','mqsittsfrliii','mciinsrailjllltg','mqsitttikiaiiit','micsirailt','mtsiiiqiikiai','miiiiiliit','mliiltit','miiiiliqiiill','niqiiiliji','iiniiaiittii','nqiiliaiisi','nyaiiiyii','tjiiaiiisr','ijmia','ljaiiiiiiisi','liiiiiitjiaiisir','lvaiisitir','lvqseiratteiii','lvqiitrlciki','lvqiitiiiiisi','lvqirtittliiitiaiil','pcsremi','lzqsitilil','ptiliiiti','ivoltotl','ijkjlonfo','lztiilsirgting','ptiisiair','lztitiidit','miinbul','ruitiljliiii','ssiiaiiiit','isijilii','isiiiinizai','isiiiiiyjili','isiiiitiiiliiit','iqiititlliiti','isiqsititilji','isiliaiiinsir','isiiiiaiijia','isillaitnii','isiilittgjii','rliailili','iiailttin','riiiajiaiiin','lisitiii','tialii','riiainaili','rliaiiisiraiil','rliairlii','rliciiljai','rlitisliiiisfa','rlikiiliiii','rlicirljaiiilsr','rlictiliaiwai','rlicitcsk','rliiitijik','rliiititlii','rliilisirai','rliiiijiiisi','rliiiliilj','rlioiisrkait','ttiai','litilaittsr','litittiljtll','liaiiteo','lilai','uisitiis']
F3_D2 = ['abu','ailr','akar','anak','anglin','anjiiing','apa','apii','asap','awan','ayah','bagaiimana','bagus','baiik','bakar','baliik','banyak','bariing','baru','basah','batu','beberapa','benar','bengkak','benlih','berat','berburu','berdiirii','berenang','berii','berjialan','berkata','berkelahii','besar','biilamana','biinatang','blintang','buah','bulan','bulu','bunga','bunuh','buruk','burung','busuk','caciing','ciium','cucii','dagiing','dan','danau','darah','datang','oaun','oebu','dekat','oengan','dengar','oiingiin','dorong','dua','ouduk','ekor','empat','engkau','galii','garam','garuk','gemuk','gligii','giigiit','gosok','gunung','hantam','hapus','hatii','hiidung','hiidup','hiijiau','hilsap','hiitam','hiitung','hujian','hutan','la','lbu','lkan','lkat','lnii','lsterii','ltu','jahiit','jantung','jatuh','jauh','kabut','kakii','kalau','kamii','kamu','kanan','karena','keciil','kepala','keriing','kiirii','kotor','kuku','kuliit','kuniing','kutu','laiin','laneiit','laut','lebar','leher','lelakii','lempar','liiciin','liidah','liihat','liima','ludah','lurus','lutut','maiin','makan','malam','mata','mataharii','matii','membelah','mengaliir','mengapung','meniikam','merah','mereka','miinum','mulut','muntah','nama','nantli','napas','nyanyii','orang','pada','panas','panjiang','paslir','pegang','pendek','peras','perempuan','pergli','perut','piikiir','pohon','potong','punggung','pusar','putiih','rambut','rumput','saklit','satu','saya','sayap','sediikiit','sempiit','semua','sliang','siiapa','suamii','sungaii','tahu','tahun','tajiam','takut','talii','tanah','tangan','tariik','tebal','tellinga','telur','rerbang','tenawa','tetek','tlidak','tiidur','tiiga','tiipiis','tiiup','tongkat','tua','tulang','tumpul','uang','ular','usus']
F4_D2 = ['abu','alir','akar','anak','anglin','anliling','apa','apli','asap','awan','ayah','bagalimana','bagus','balik','bakar','baliik','banyak','barling','baru','basah','batu','beberapa','benar','bengkak','benlih','berat','berburu','berdiirli','berenang','berli','berlialan','berkata','berkelahli','besar','blilamana','blinatang','biintang','buah','bulan','bulu','bunga','bunuh','buruk','burung','busuk','cacling','clium','cucli','dagling','dan','danau','darah','datang','daun','debu','dekat','dengan','dengar','dlinglin','dorong','dua','duduk','ekor','empat','engkau','galli','garam','garuk','gemuk','gligli','gliglit','gosok','gunung','hantam','hapus','hatii','hlidung','hlidup','hiiliau','hlisap','hlitam','hiltung','hujian','hutan','la','lbu','lkan','lkat','lnii','lsterli','ltu','jahlit','jantung','jatuh','jauh','kabut','kakli','kalau','kamli','kamu','kanan','karena','keclil','kepala','kerling','klirli','kotor','kuku','kullit','kunling','kutu','laiin','langlit','laut','lebar','leher','lelakli','lempar','lliclin','llidah','llihat','llima','ludah','lurus','lutut','malin','makan','malam','mata','mataharli','matli','membelah','mengaliir','mengapung','menlikam','merah','mereka','mlinum','mulut','muntah','nama','nantli','napas','nyanyli','orang','pada','panas','panjiang','pasiir','pmang','pendek','peras','perempuan','pergli','perut','pliklir','pohon','potong','punggung','pusar','putlih','rambut','rumput','saklit','satu','saya','sayap','sedliklit','sempiit','semua','sliang','sliapa','suamli','sungali','tahu','tahun','taliam','takut','talli','tanah','tangan','tarlik','tebal','tellinga','telur','terbang','tenawa','tetek','tlidak','tlidur','tiiga','tliplis','tliup','tongkat','tua','tulang','tumpul','uang','ular','usus']
F5_D2 = ['abu','aipr','aka','anaki','angipn','anjptpng','apa','apii','asap','awm','aym','bagaipmma','bagus','baipk','bakx','baijpk','bmyak','batiing','ljciu','ijiitspaih','ljaitu','lijcibuipraipiav','bipnair','beingkakp','bentph','bcirat','bgprburu','betdtprtp','berenang','berip','betjialm','berkata','betkeiahip','bescv','biilmma','jiiittipitiittlg','ijiitttiiitijfv','litiivtii','ijtiliitti','ijtilti','iitingpai','ljunuh','iiiurtik','iiiiurungp','busiuk','cfacipng','cipum','cfucii','dagipng','ijan','dianau','dam','daag','daun','debu','dekat','dengm','dengx','dipngipn','dorong','dua','duduk','ekor','fvmpat','tpnglpau','ciaiip','garam','garulp','gcmttlp','gpiigtp','gtpgiit','gosfiplp','ctutiiitjgi','iiiiiutipiiti','hapus','liatip','htpdung','htpdup','laiijpau','htisap','htptm','hiptunpo','hupiim','hutm','la','ibu','tkm','iliat','inip','tsterip','itu','jmipt','jmtung','jamh','jauh','kabut','kwii','kaiau','kamip','kamu','kmm','kaena','kecipi','kepaia','kertpng','kiprtp','krotot','ktilpu','kuiiit','kuniing','kutu','laiin','lmgipt','laut','lebar','leher','leiakip','lempar','liicpiin','lipdm','liphat','lipma','ludah','lurus','lumt','maipn','makm','maiaim','mata','mataharii','matip','miembeiah','mengaiipr','mengapung','menjpkm','metm','mereka','miinum','muiut','muntah','nma','nanttv','napas','nyovyii','ormg','pada','paoas','panpipaingi','pastir','pewwmww','pendetp','pcwrasp','ijuprempuavn','pergtv','ipcrut','piiikipr','pohon','potong','punaung','pusa','puttph','rambut','rumput','salpipt','satiu','saya','sfayap','sedipktpt','iscimpipt','spemua','isivmg','isijiapa','isimip','isiunpi','iidiu','iidiun','iifaiplpaiin','takut','ilpaltp','tanah','ifangan','tariik','tebal','iteiiinga','ireiur','terbang','tenawa','tetiek','tipdak','tipdur','tipga','tippips','tipup','irongliat','tua','tfuimg','ilpumpui','uang','tilapr','tjisiuisi']

F1_D3 = ['abu','aiir','akar','anak','anaiin','anjiiing','apa','apii','asap','awan','ayah','aaqaiimawa','bagus','eaiik','eakar','ealiik','banyak','eariing','baru','msah','bau','eeberapa','eenar','bewgkak','beniih','aerat','eerburu','eerdiirii','berenang','eerii','berjialaa','berkah','aerkelahii','sesar','biilamana','biinahna','biinhwq','buah','eiulan','eulu','eunga','bunuh','euruk','buruno','eusvk','caciins','ciiun','cucii','oasiing','oan','oanau','oarah','oaians','oaun','oesu','oekat','oenaan','oensar','oiingiio','ooronq','oua','ouduk','ekor','enpai','enqkau','ealii','mran','carvk','eenuk','ciisii','eiiaiit','eosok','cunung','hanmn','hapus','mni','hriunq','hiidup','hiijiau','hiisap','himim','hiitunq','hupiw','huhn','la','ibu','lmn','lmt','inii','iaerii','itu','mhri','jantung','jmh','uauh','obr','mii','gau','miii','kamv','kawan','onna','keciil','kepala','keriins','kiirii','kotor','kuku','kuliit','kuniina','kutu','laiin','lanqiit','laut','lmar','lmer','lelakii','lompar','uiiciin','uiidah','liihat','liima','uudah','uurus','umi','wiin','mo','valan','mae','mmmwi','mtii','mah','vmaiiir','mewwm','menimn','mwh','uers','mmimi','wlu','umh','naw','nantii','napm','nymii','orami','pada','pans','panpio','psir','pom','pmi','peras','perempuan','pergii','perut','piikiir','pohon','poiona','punpgung','pusar','putiih','rambui','rumput','sakiit','satu','saya','sayap','sediikiit','sempiit','semua','siianq','siiapa','suaoii','sungaii','tahu','tahun','taiiam','takui','talii','tanah','tanoan','tariik','tebal','teliinga','relur','terbana','tedawa','tetek','tiidak','tiidur','tiiga','tiipiis','tiiup','tongkat','tua','tulanq','tumpul','uang','uiar','usus']
F2_D3 = ['iailw','air','iiiuari','aezli','ruwiiin','iqajiwi','w','mii','imr','iaivo','aivim','siw','imsiniisi','imai','sii','miaii','misk','oi','sirii','uiiisimi','iuiitq','siilriiiqla','siiiqr','biiiiisisa','iliiiiiwi','siwt','riirlvirq','bqiirdiirii','siiiviiiqwi','nirii','krjijiaii','ksiqiq','sirliiilqiliii','siiwi','hjisiwiq','hiiwwi','biiiiiw','biislii','bijswi','uiiuii','hiiop','uiiaiili','uqnli','biriiie','biiisiiik','tiisiiiiv','tziiiqvi','tiiiiqiii','liiqiizi','ijqiii','ljvasii','mi','oi','ma','d','tsi','wi','mw','ljiimiii','lixrivisr','lixiq','iijiidia','fixiir','iiiiiqiiai','fiiiiwiiii','qiiqlii','riisi','itvimqi','tiiininiiqi','tiipiii','iqiipiii','irioiviik','qiiqniiv','hqiisni','hqqis','liqtii','hiiiivazi','hiiiiup','hiijiqiv','lliislv','liiiiao','hiiivwz','hiijioi','hurzin','izi','im','iuin','iuii','ioii','iiisiqviii','io','jaiiii','jqaniw','jiqqui','jaiih','kqliiii','kmi','km','mnii','miii','mo','kqrqioq','keiriil','krilas','kcriioz','giii','koior','kiirii','kvuit','kvmivi','kviivi','miii','mwii','mut','uso','uhrir','slmii','lziiiiiw','lziqiiiw','liiiiih','liiijiqi','iliioqi','lvdiili','liiriiisi','liiiiir','niiiin','nmio','naoi','nqiizi','namoii','iistqui','neoikiiliiii','neapliiii','niriwiizlvw','ineiiuiiii','noqli','uqircikq','niiiimv','nqilqi','niioiji','iniisiaq','nisniii','iniizipsi','iiiinyqiiijvi','tjwvi','lsiiq','ivavss','iiaivijiwr','rsiri','pvosr','rqinrlqili','reraii','prirciviipuzin','pqirpi','pqiinvt','riimi','rohoa','lvotioiwi','pqnsinnzi','miqir','rutai','mbut','rviiiizvl','isiqkiit','istu','isiqviiq','isiqyiqii','isiiiiiiiikiii','iqiiiiiliiiii','iqiiiiiiia','isiiiavr','isiuiiiivq','isiiziiiiii','isionsnii','tqhii','iiiaivii','liiajiaiiii','iliaiqi','nijiii','nioqli','iiqinsrqa','tiariik','zrqiilm','iliqiiiiiiimi','niiiiiir','iiiiirliqsr','iliqinqvq','rliiiiiqiili','iliiiiizil','iliiiiliir','rlisis','riiiilviiisi','rliiiiiiv','niiiiiziliqii','liiiai','tiilqiv','rriiiiiliviii','uaiisir','liilqr','liiisiiiisi']
F3_D3 = ['wiiii','wii','mii','iim','mii','w','w','imii','siii','mi','w','siaws','aagus','mi','is','si','wii','o','srv','easah','satu','aeserapa','aewar','sewsvak','saiiw','srra','eersvrv','aerdiirii','svrewawe','aerii','aeiiaian','aerkas','aewiahii','asrr','aiiianawa','aiiwasawz','uiintane','eqah','svlan','aviv','svnza','avnvh','svrvr','evrvwz','avsvk','aciife','ciivn','cvqii','oaziin','oar','oanav','iiwi','ioiii','iiwiiiii','im','siii','qi','wwsar','oiiwiin','mrov','ova','nrvk','ekor','eopai','eozkav','eaiii','earan','srvk','eenur','siisii','siisiit','sosok','svwvns','hantan','hapus','hatii','hiidvws','hiidup','hiiliav','hiisap','hiitan','hiixvos','hqiaw','hvian','ia','isv','ixao','ixar','inii','isxerii','itv','jahiit','jawtqns','jatuh','jauh','kabvt','kakii','kzlav','kanii','kanv','kawsn','wrefa','keciii','kepala','keriinz','kiirii','koior','xvkv','kviiit','kvoiinz','kvtu','siif','laosqi','laut','lqsqr','lqhqr','telaki','lqnpar','liieiw','liidah','lihar','lina','lvdah','lurus','lutut','maiiw','makaw','vaian','vara','matzhazi','mati','vsnseiah','mewzaiir','mewsapvws','vewimn','mezah','verqka','viirvn','vvivt','vvwsah','nsna','nantii','napav','nyarvii','oraws','fada','pawas','fanjians','fasiir','fezawe','feooek','feras','ferenpvaw','pereii','pest','fikiir','fohow','foxonz','fvwezqns','fvsar','fvsiiw','masw','avwwi','sakiit','svtv','saya','sayap','seriikiii','senpiit','senva','siiawz','siirpa','svvnii','svnzaii','rahv','trhvn','tajian','tavr','toiii','taoah','rawzao','tariiq','tesal','triiiwza','reivr','trrsoz','tenawa','reer','riidak','riidvr','riza','tiipiis','riivp','roozxat','rva','tvianv','tvnpui','uang','uiar','usvs']
F4_D3 = ['abu','aiir','akar','anak','adgiin','anjiiino','apa','apii','asap','awan','ayah','bagaiimana','bagus','baiik','eakar','baliik','banyak','bariing','earu','bamh','batu','beuerapa','benar','bengkak','beniih','berat','berburu','berdiirii','berenang','berii','eeriialan','bermm','berkelahii','bemr','biilamana','bvinamnq','sinhng','buah','buian','bulu','bunga','bunuh','euruk','burung','eusuk','caciing','ciiumi','cucii','dagiing','dan','oarah','oarah','oamng','daun','oebu','oekat','oenqan','oengar','oiingiin','mrong','dua','ouduk','ekor','emmt','engkau','galii','earam','oaruk','emuk','giigii','eiigiit','eosok','gunum','hantami','haws','hatii','hiiovng','hiidup','hiijiau','hiisap','hiihn','hiiung','huiian','hmn','la','lm','ikm','ikat','inii','iseni','m','jahri','uotm','jauh','jm','kabr','kakii','kalau','kamii','kamu','kanan','karena','keciil','kepaia','keriing','kiirii','kotor','kuku','kuliit','kuniing','kutu','miio','mngiit','mut','lebar','leher','lelakii','lompar','uiiciin','liidah','liihat','liima','uudah','lurus','lutut','maiin','uakan','malam','mata','mataharii','vatii','vembelah','menpaliir','mengapune','meniikam','merah','mereka','miinun','mulut','muntah','nama','nantii','nams','nyanyii','orang','padz','panas','panjiang','pasiir','pegang','pendok','peras','perempuan','pergii','perut','piikiir','pohon','potong','pungqung','pusar','putiih','hambr','rumput','sakiit','satu','saya','sayap','sediikki','sempit','semua','siiang','siiapa','suamli','sungaii','tahu','tahun','tajim','takr','talii','tanah','tangan','tariik','teml','teiiinga','telur','termng','temwa','tetek','ridak','tiidur','tiiga','tiipvis','tiiup','tongkat','tua','tuiang','tumpul','uang','uiar','usus']
F5_D3 = ['ahv','aiir','mii','aoak','msiin','aitjiiiaz','apia','apii','asiar','awm','ayah','niagqimsnq','ibqsriiisi','biaiik','bami','baliik','boys','baning','bo','bsah','mn','besrapa','bma','aengm','beawi','berai','berbun','berdiini','beromz','berii','bsirjialm','berkaiiai','berkrlmii','hieso','biijmaoq','biims','biiawe','bum','aulm','bqlu','bvaga','bmm','bvruk','burune','buisuk','csizioz','ciiiun','cvqii','daziing','uo','doav','dsa','datog','oqqa','desv','delir','ioeasoi','uenss','pvioziin','oorons','oua','mduk','emr','enpaii','fiiiow','oaiii','czaao','cmk','cziqmji','ciipi','oviziii','cosok','ovwvosr','laom','hsfvisi','liatii','hiidmz','hiidivp','ihiiiiiav','fiiisq','iiiim','hiims','tiqjio','hum','ja','tbu','lkm','ikat','tnii','tserii','ts','jmiit','jmiqos','janh','jauh','kasvi','gii','kiaaq','koii','mv','kmao','ksqm','keciil','kspala','kenios','kvini','kotor','kvm','kuliit','kvoiiaz','kqm','iiain','lmgiit','laqi','mbs','leher','lelakii','tzenps','liiciin','liidm','liiha','liiina','liqdah','luns','lqat','maiio','makm','mdo','mqitq','mauhsii','miqtii','menbeijqh','mqagaliir','mensqpvag','mqaiiko','merah','mqireka','viinuo','mulvi','viaiitah','nma','nianni','siafas','nyiosriii','mme','paua','roas','poiiiiqing','psiir','pesos','poidek','po','peropqo','pergii','peni','pmii','pohow','poionz','msmz','psss','puiiia','mnsvt','rvnpqt','ssiit','isas','sajia','isayaf','sqdaiiit','siewpiii','sqnva','isiiiog','siiva','svoii','smzaii','tav','timva','tajim','tmi','taii','toa','tow','toik','teba','teliiap','teivr','tasos','twvs','teer','tiida','tiidw','tiiza','tiipiis','tiiqp','tmfat','tw','rvios','tvnml','umg','ms','usw']

F1_D4 = ['abu','aiir','akar','anak','angiin','anjiiing','apa','apii','asap','awan','ayah','bagaiimana','bagus','baiik','bakar','baliik','banyak','bariing','baru','basah','batu','beberapa','benar','bengkak','beniih','berat','berburu','berdiirii','berenang','berii','berjialan','berkata','berkelahii','besar','biilamana','blinatang','biintang','buah','bulan','bulu','bunga','bunuh','buruk','eurung','busuk','caciing','ciium','cucii','dagiing','dan','danau','darah','datang','daun','debu','dekat','dengan','dengar','diingiin','dorong','dua','duduk','ekor','empat','engkau','galii','garam','garuk','gemuk','giigii','giigiit','gosok','gunung','hantam','hapus','hatii','hlidung','hiidup','hiijiau','hiisap','hiitam','hiitung','hujian','hutan','la','lbu','lkan','lkat','lnii','lsterii','ltu','jahiit','jantung','jatuh','jauh','kabut','kakii','kalau','kamii','kamu','kanan','karena','keciil','kepala','keriing','kiirii','kotor','kuku','kuliit','kuniing','kutu','laiin','langiit','laul','lebar','leher','lelakii','lempar','liiciin','liidah','liihat','liima','ludah','lurus','lutut','maiin','makan','malam','mata','mataharii','matii','membelah','mengaliir','mengapung','meniikam','merah','mereka','miinum','mulut','muntah','nama','nantii','napas','nyanyii','orang','pada','panas','panjiang','pasiir','pegang','pendek','peras','perempuan','pergii','perut','piikiir','pohon','potong','punggung','pusar','putiih','rambut','rumput','sakiit','satu','saya','sayap','sediikiit','sempiii','semua','siiang','sliapa','suamii','sungaii','tahu','tahun','tajiam','takul','talii','tanah','tangan','tariik','tebal','teliinga','telur','terbang','tedawa','tetek','tiidak','tiidur','tiiga','tiipiis','tiiup','tongkat','tua','rulang','tumpul','uang','ular','usus']
F2_D4 = ['niu','aiir','mai','ztilai','iiltfiin','mijiiinz','iniljai','apii','asiap','aiwai','ajiah','bgiaimzinzi','baipiiisi','bari','baiiq','bjiiik','bainyai','boiiingi','bcili','baisijil','bqiiiu','miieraljai','hiina','bqiiwkai','biiioiih','bqiirait','bcirliuru','berdiirii','mircinoip','mrii','beirjidain','kiirkziui','bcirkclmii','rvisiar','biilaimianai','biiinatami','biintaiiig','biijit','bulan','bulii','btinziz','bunuli','burqik','burtlilg','ibiislai','ciiaciiingi','ciiiiilin','csucii','iiijsiiriiivi','ijaii','iijainaili','diziirdi','ijaiuiop','ljaitin','iijcbu','ijckaiti','ijeinzan','ljciwiu','ijiiiipin','ijiirotisr','ijiizi','ijiiiliik','iiiikqir','liiiiiipi','liiiitfkaiii','ciiailii','tiiiciaiil','iviairuki','tiiqiimlik','iiisini','tziiifiiii','tiiosik','triuiliinsri','hantan','hailjiiisi','hzitii','fltiqlutlzi','hiidiip','hiijiaiii','hiisiap','hiiuim','hiitilng','hujioi','huuii','iai','ibu','imiii','lkat','lnii','lisitqirii','itiii','jjiiiiii','jziiituiisr','jaiiiit','jqiiili','kzilzui','qiii','gaiii','kimiii','mlu','oiiain','orcinai','keicdi','kciijjiai','keriiog','kiiirii','kokzr','kuku','kuliii','kiiiiiing','ikiitii','laiiiv','lziiisriii','liul','iziliair','iziiihqviii','uiiilaikii','lziinlizai','liiqiiin','liiiljii','lqihaiti','iiiiiiia','lurlaili','iiiiruis','lutiiii','mqiiin','naiain','maan','nau','namiariii','natii','sietitivqvlaih','inleilmiiir','mcitpiaijiiitg','mciiiikiqiin','vqiiiizih','niiiiiqikai','iniinqiiti','miiliiii','niiiniqih','niiiaiitiai','inaiitii','nailiaisi','inivaiiiyiii','cijraiitsr','ivziilai','lvainaiisi','liaiitjiaiitgi','paiisiiir','pcpansr','iiqisitirlqiik','izqiiiraiis','pqvirqiinijiiazt','pergii','pcniii','pmiir','ljoitoii','piztqiilgi','pungriig','pliisiair','lzutiih','mntbut','riuililvliiii','isisiit','isiaiiili','isiaiivai','isiiayap','sqiiiiiqikiiiii','iseitnljlii','iqiitliiai','isiiaw','vsiiialva','isiiiainii','isiiinpiai','taihqi','rlidun','rraijiaitn','iiisqiiii','iiailii','riiainaill','riiairzio','rliairiik','iliqiiliji','rliqiliinpa','rrelir','rlicrlzmisiri','qienawai','rliqiiiicik','rriirlaik','riiiiduii','tsiria','tiiiijiiis','iiiiliij','tongkiati','niua','liulaimr','iliuiiipm','uainfi','ulair','liisiliisi']
F3_D4 = ['abu','aiir','akar','anak','angiin','anjiiing','apa','apii','asap','awan','ayah','bagaiimana','bagus','baiik','bakar','aaliik','sanyak','eariine','baru','basah','batu','beberapa','benar','bengkak','beniih','berat','berburu','serdiirii','berenang','aerii','berjialan','berkata','aerkelahii','aesar','biilamana','biinatang','biintang','buah','bulan','bulu','eunga','bunuh','auruk','burung','busuk','caciing','ciium','cucii','oagiing','oan','oanau','oarah','oatane','oaun','oebv','oekat','oenzan','oenzar','oiineiin','oorons','oua','ouduk','ekor','empat','enekau','eaiii','earam','earuk','gemuk','eiigii','giigiit','gosok','gunung','hantam','hapus','hatii','hiidurg','hiidup','hiijiau','hiisap','hiitam','hiitung','hujian','hutan','la','lbu','ikan','ikiqt','inii','lsterii','itu','jahiit','jantung','jatuh','jauh','kabut','kakii','kalau','kanii','kamu','kanan','karena','keciil','kepala','keriinz','kiirii','kotor','kuku','kuliit','kuoiine','kutu','laiin','lanziit','laut','lebar','leher','lelakii','lempar','liiciin','liidah','liihat','liima','ludah','lurvs','lutut','maiin','makan','malam','mata','mataharii','matii','membelah','menzaliir','meneapuwe','meniikan','merah','mereka','miinum','mulut','muntah','nama','nantii','napas','nyanyii','orang','pada','panas','fanjiang','pasiir','fegane','pendek','peras','ferempuan','pergii','ferut','piikiir','fohon','potowg','fungguoe','fusar','putiih','rambut','rumput','sakiit','satu','saya','sayap','sediikiit','sempiit','semua','siiane','siiapa','suamii','sungaii','tahu','tahun','rajiam','rakut','ralii','tanah','tangan','tariik','tebai','teiiinea','telur','rerbane','reeawa','tetek','riidak','tiidur','riiga','tiipiis','tiiup','tongkat','rua','rulane','tumpul','uang','ular','usus']
F4_D4 = ['abu','aiir','akar','anak','angiin','anjiling','apa','apii','asap','awan','ayah','bagaiimana','bagus','baiik','bakar','baliik','banyak','bariing','baru','basah','batu','beberapa','benar','bengkak','beniih','berat','berburu','berdiirii','berenang','berli','berjialan','berkata','berkelahii','besar','blilamana','blinatang','biintang','buah','bulan','bulu','bunga','bunuh','buruk','burung','busuk','caciing','ciium','cucii','dagiing','dan','danau','darah','datang','daun','debu','dekat','dengan','dengar','diingiin','dorong','dua','duduk','ekor','empat','engkau','galii','garam','garuk','gemuk','gligii','giigiit','gomk','gunung','hantam','hapus','hatii','hiidung','hlidup','hiijiau','hiisap','hiitam','hiitung','hujian','hutan','la','lbu','lkan','lkat','lnii','lsterli','ltu','jahiit','jantung','jatuh','jauh','kabut','kakii','kalau','kamii','kamu','kanan','karena','keciil','kepala','keriing','klirii','kotor','kuku','kuliit','kuniing','kutu','laiin','langiit','laut','lebar','leher','lelakii','lempar','lliciin','liidah','liihat','liima','ludah','lurus','lutut','maiin','makan','malam','mata','mataharli','matii','membelah','mengaliir','mengapung','meniikam','merah','mereka','miinum','mulut','muntah','nama','nantii','napas','nyanyii','orang','pada','panas','panjiang','pasiir','pegang','pendek','peras','perempuan','pergii','perut','piikiir','pohon','potong','punggung','pusar','putlih','rambut','rumput','sakiit','satu','saya','sayap','smlikiit','sempiit','semua','sliang','siiapa','suamii','sungali','tahu','tahun','tajiam','takut','talli','tanah','tangan','tarlik','tebal','teliinga','telur','terbang','tenawa','tetek','tildak','tlidur','tliga','tlipiis','tiiup','tongkat','tua','tulang','tumpul','uang','ular','usus']
F5_D4 = ['abu','atir','aks','anaik','angiin','anjiiing','apa','apii','asap','awim','aym','bagaiimoa','bagus','baiik','bakar','batik','bmysi','baiiing','bo','bsah','batu','bebcrapai','bqinar','bengks','bciniih','berat','berbum','berdiirii','berenang','berii','berjiam','berkata','berkelmti','besar','biilamma','biinatang','biintmg','bud','bulm','bulu','bunga','bunuh','buruk','burung','busuk','caciing','ciium','cuqiii','dasiing','dan','dmau','dso','datmg','daun','debu','dekat','densm','denga','diineiin','doroog','dua','duduk','eiikor','enpat','engkau','czalii','osm','ccuk','oremuk','cziisii','oiigiit','oosok','oiunung','hoitaim','haipuis','hatii','iliidung','hiidup','hiijiau','iiltisap','hiitam','hiitung','hujim','hutan','ia','ibu','ikm','ikat','lnii','lsterii','ltu','jdiit','jmtung','jatuh','jauh','kasui','kakii','kalau','kanii','kamu','kmm','karena','keciil','kepala','kciriing','kiirii','kotor','kuku','kuliit','kuniins','kum','laiiin','logiit','tiiziut','lebs','iveher','lelsii','lempar','liiciio','liidm','liihat','liima','iiudm','tiuns','lutut','maiin','makm','maiam','mau','mataharii','matii','menbeld','mengaliir','mengapung','meoiikam','merd','mereka','miinum','mulut','muam','noa','nantii','naps','nymyii','orms','pada','panais','pmjiog','psiir','pegoe','peodek','pssrsi','perempum','pergii','pemt','piikiir','pohon','potoing','punggung','pusa','putiih','rambut','runput','sakiit','satu','saijiai','sayap','sedtikiit','sempiit','semua','siiang','siiaipa','sumii','sunsaii','tmu','tmun','niajiam','takut','talii','iraindi','tangoi','liaiik','tqvbail','teliinga','telur','terbang','tenawa','tetek','tiidak','riidur','tiiga','tiipiis','tiiup','tongkati','irua','tulang','tunpul','umg','ula','usus']

F1_D5 = ['abu','airr','akar','anak','anglin','anjiiing','apa','apli','asap','awan','ayah','bagalimana','bagus','balik','bakar','ballik','banyak','barling','baru','basah','batu','beberapa','benar','bengkak','beniih','berat','berburu','berdlirii','berenang','berli','berjialan','berkata','berkelahli','besar','blilamana','biinatang','biintang','buah','bulan','buiu','bunga','bunuh','buruk','burung','busuk','caciing','clium','cucii','dagiing','dan','danau','darah','datang','daun','debu','dekat','dengan','dengar','dlinglin','dorong','dua','duduk','ekor','empat','engkau','galli','garam','garuk','gemuk','gligii','giigiit','gosok','gunung','hantam','hapus','hatli','hiidung','hiidup','hlijiau','hiisap','hlitam','hlitung','hujian','hutan','la','lbu','ikan','lkat','lnli','lsterli','ltu','jahlit','jantung','jatuh','jauh','kabut','kakii','kalau','kamil','kamu','kanan','karena','kecilil','kepala','kerling','klirli','kotor','kuku','kuliit','kuniing','kutu','lalin','langlit','laut','lebar','leher','lelakli','lempar','lliclin','llidah','llihat','llima','ludah','lurus','lutut','maiin','makan','malam','mata','mataharli','matli','membelah','mengallir','mengapung','menlikam','merah','mereka','mlinum','mulut','muntah','nama','nantli','napas','nyanyli','orang','pada','panas','panjiang','paslir','pegang','pendek','peras','perempuan','pergrl','perut','pliklir','pohon','potong','punggung','pusar','putiih','rambut','rumput','saklit','satu','saya','sayap','sedliklit','semplit','semua','sliang','sliapa','suamli','sungali','tahu','tahun','tajiam','takut','talli','tanah','tangan','tarlik','tebal','tellinga','telur','terbang','tenawa','tetek','tlidak','tlidur','tliga','tirplis','tliup','tongkat','tua','tulang','tumpul','uang','ular','usus']
F2_D5 = ['iqu','mir','ki','hia','mpin','ihijiqi','aim','apli','miap','aiwn','aym','bpitnma','bigss','bmr','bsn','bdri','bmiyri','bamig','bntl','bmm','batui','behrapa','benn','bcvwks','bcintilt','bciriailii','iberbuu','mrmirti','bercivnmg','bierli','rdidmi','bermm','berkelmii','besai','blilmima','bmiamw','blinmg','bwit','btilan','bmu','bunp','bimd','biuid','burum','bustl','cacting','cvliunl','ctlcli','dging','dan','ljainaitt','idmi','dam','daiun','dcibtt','dekai','denp','diengn','dlinprn','dorow','dua','dudq','mor','empat','fvfwkaui','czdti','czum','gnnuiik','cvctnuk','gilipii','ggiit','mssok','gmtmg','fimtmn','hapus','haitii','htidung','hiidup','hiiijiiaiu','hlrsap','hiiimi','hlitung','hittjim','hum','ia','ibu','is','lmt','inli','lstcirli','lm','jdirt','jmtung','jatuh','jad','obut','mi','gatt','mjr','kmiu','kmm','knena','kccdi','kcipda','kiciirmig','qrlr','kotor','kwu','ktdlit','kmmiig','kum','lmi','lmglrti','lauti','mba','mhcr','mlki','dpm','llioiin','liidd','lmiat','liima','ludah','lum','lutut','mairn','mwiian','mdm','mam','maimitarli','maui','micvmbcslah','memri','miegptingi','menwin','micid','mcrekia','mlinm','mmtlt','mttntm','nma','nmitli','napmii','nyanpi','orq','piaitilaii','pmas','painjimg','pmri','pcgmg','pendek','perm','pierempuan','pcrgli','pcmili','pwri','ljohon','potom','pmgiong','puisa','pttdi','mbut','rmput','qsikit','qsatu','saya','sayiap','ikidlihrt','vseinplit','isemuiia','silimgi','sliap','vsuainti','isungiati','timiu','tidm','taijim','qisiut','tdti','qiiianaih','tmgian','turr','tebd','tcislmiga','teluri','rfcirbang','qiienawa','tcistek','ttidq','ttidur','tiip','tiiptiis','tiiup','tonoat','ttta','tmq','rrumptd','uaing','ulu','ustls']
F3_D5 = ['abu','aiir','akar','anak','anglin','anjiiing','apa','aprl','asap','awan','ayah','bagailmana','bagus','balik','bakar','ballik','banyak','barlrng','baru','basah','batu','beberapa','benar','bengkak','benilh','berat','berburu','berdlirli','berenang','berii','berjialan','berkata','berkelahii','besar','blilamana','biinatang','blintang','buah','bulan','bulu','bunga','bunuh','buruk','burung','busuk','cacling','clium','cucil','dagling','dan','danau','darah','datang','daun','debu','dekat','dengan','dengar','diinglin','dorcng','dua','duduk','ekor','empat','engkau','galii','garam','garuk','gemuk','gligli','gliglit','gosok','gunung','hantam','hapus','hatli','hlidung','hiidup','hiijiau','hlisap','hiitam','hiitung','hujian','hutan','la','lbu','lkan','lkat','lnli','isterii','ltu','jahilt','jantung','jatuh','jauh','kabut','kakli','kalau','kamli','kamu','kanan','karena','keciil','kepala','keriing','klirli','kotor','kuku','kullit','kuniing','kutu','laiin','langiit','laut','lebar','leher','lelakli','lempar','lilclin','llidah','liihat','llima','ludah','lurus','lutut','maiin','makan','malam','mata','mataharii','matii','membelah','mengaliir','mengapung','meniikam','merah','mereka','miinum','mulut','muntah','nama','nantli','napas','nyanyii','orang','pada','panas','panjiang','paslir','pegang','pendek','peras','perempuan','pergli','perut','piiklir','pohon','potong','punggung','pusar','putiih','rambut','rumput','saklit','satu','saya','sayap','sediikiit','sempiit','semua','siiang','siiapa','suamii','sungaii','tahu','tahun','tajiam','takut','talii','tanah','tangan','tariik','tebal','teliinga','telur','terbang','tenawa','tetek','tlidak','tlidur','tiiga','tlipiis','tiiup','tongkat','tua','tulang','tumpul','uang','ular','usus']
F4_D5 = ['abu','alir','akar','anak','anglin','anliiing','apa','apli','asap','awan','ayah','bagalimana','bagus','balik','bakar','ballik','banyak','barling','baru','basah','batu','beberapa','benar','bengkak','benlih','berat','berburu','berdlirli','berenang','berli','berlialan','berkata','berkelahli','besar','blilamana','blinatang','blintang','buah','bulan','bulu','bunga','bunuh','buruk','burung','busuk','cacling','clium','cucli','dagling','dan','danau','darah','datang','daun','debu','dekat','dengan','dengar','dlingiin','dorong','dua','duduk','ekor','empat','engkau','galli','garam','garuk','gemuk','gligli','giiglit','gosok','gunung','hantam','hapus','hatli','hlidung','hlidup','hiljiau','hlisap','hlitam','hrltung','hujian','hutan','la','lbu','lkan','lkat','lnli','lsterli','ltu','jahiit','jantung','jatuh','jauh','kabut','kakli','kalau','kamli','kamu','kanan','karena','keclil','kepala','kerling','klirli','kotor','kuku','kullit','kunling','kutu','lalin','langlit','laut','lebar','leher','lelakli','lempar','lliclin','llidah','llihat','liima','ludah','lurus','lutut','malin','makan','malam','mata','mataharli','matli','membelah','mengallir','mengapung','menlikam','merah','mereka','mlinum','mulut','muntah','nama','nantli','napas','nyanyli','orang','pada','panas','panliang','paslir','pegang','pendek','peras','perempuan','pergli','perut','pliklir','pohon','potong','punggung','pusar','putlih','rambut','rumput','saklit','satu','saya','sayap','sedliklit','sempiit','semua','sliang','sliapa','suamli','sungali','tahu','tahun','tajiam','takut','talli','tanah','tangan','tarlik','tebal','tellinga','telur','terbang','tenawa','tetek','tlidak','tlidur','tliga','tliplis','tliup','tongkat','tua','tulang','tumpul','uang','ular','usus']
F5_D5 = ['abu','atir','mn','mak','mgtin','mjiling','apa','apti','asap','awm','ayd','bagalimma','bagus','balik','bwa','baltik','bmyw','bniing','bm','bmd','bam','beberapa','bena','benaak','benhi','berat','berbum','bcrdtirli','berenmg','berti','berjialm','berkata','berkelahtr','besa','blilmma','btrnatmg','blinmg','bum','bulm','bulu','bunga','bunuh','bumk','bumng','busuk','csiing','ctium','cucti','dagting','dm','dmau','dnm','damg','daun','debu','dekat','dengm','denga','dlinglin','dorong','dua','duduk','ekor','empat','engmu','galli','garm','gamk','gemuk','giigii','gtiglit','gosok','gunung','hmtam','hapus','hatli','hiidung','htidup','hlijiau','htisap','htitm','hiimg','huiiim','hum','ia','ibu','ikm','ikat','lnti','isteni','itu','jmlit','jmmng','jamh','jauh','kabut','kwii','kaau','kmii','kamu','kmm','ksena','keclii','kepala','kerting','ktirli','kotor','kuku','kullit','kunting','kum','laiin','lmgtit','laut','lebar','leher','lelsti','lempar','liiciin','llidm','llihat','llima','ludd','lums','lumt','matin','makm','mmaim','mata','matahsii','matii','membelm','mengallir','mengapung','meniikm','merm','mereka','miinum','mulut','muntm','nama','nmtli','napas','nymyii','ormg','pada','pmas','paniiiang','pasiir','pegmg','pendek','perm','perempum','piergti','pemt','piiktir','pohon','potong','pmgmng','pusn','puttih','rambut','rumput','saktit','sam','saya','sayap','sedlimit','sempiit','semua','slimg','sliapa','sumli','sungai','tdu','tmun','tajim','tamt','talti','tmm','tmgm','tmlik','tebarl','teltinga','telur','terbmg','tenawa','tetek','tiidw','ttidur','tliga','tliplis','tliup','tongkat','tua','tuimg','tumpul','uang','ular','usus']

In [16]:
print(len(F1_D1))
for i in F1_D1:
    dist1(i)

200
abu , abu , 0 , Orbuk
aiir , air , 1 , Aek
akar , akar , 0 , Urat
anak , anak , 0 , Anak
angiin , angin , 1 , Alogo
anjiiing , anjing , 2 , Biang
apa , apa , 0 , Aha
apii , api , 1 , Api
asap , asap , 0 , Timus
awan , awan , 0 , Ombun
ayah , ayah , 0 , Amang/Ama
bagaiimana , bagaimana , 1 , Beha
bagus , bagus , 0 , Bagak
baiik , baik , 1 , Denggan
bakar , bakar , 0 , Tutung
baliik , balik , 1 , Balik/Pudi
banyak , banyak , 0 , Godang
bariing , baring , 1 , Peak
baru , baru , 0 , Baru/Imbaru
basah , basah , 0 , Bornok/Tonu/Maraek
batu , batu , 0 , Batu
beberapa , beberapa , 0 , Pigapiga
benar , benar , 0 , Sintong/Toho
bengkak , bengkak , 0 , Bongkak
beniih , benih , 1 , Boni
berat , berat , 0 , Borat
berburu , berburu  , 1 , Mamuru
berdiirii , berdiri  , 2 , Jongjong
berenang , berenang , 0 , Marlange
berii , beri , 1 , Lehon
berjialan , berjalan , 1 , Mardalan
berkata , berkata  , 1 , Marhata
berkelahii , berkelahi  , 1 , Marbada
besar , besar , 0 , Balga
biilamana , bilamana , 1 